In [93]:
import numpy as np
import matplotlib.pyplot as plt
import math 
import iop_ext as iop #artem stuff

***DEFINE CONSTANTS - EXOPLANET OF INTEREST: Kepler 80-d***

In [94]:
Lsun = 3.846e26 #Luminosity of the Sun in W
Msun = 1.9891e30 #Mass of the Sun in KG
Mearth = 5.97219e24 #Mass of Earth in KG
Rearth = 6.371e6 #Radius of Earth in m
au = 1.49598e11 #AU in m
yr = 3600*24*365 #Year to Seconds converter
a0 = 0.5 #Slope of LHE (high energy luminosity)
tsat = 1.0e8*yr #Duration of saturation phase
G = 6.67e-11 #Universal gravitational constant
pi = math.pi #pi

#PLANET: Go to NEA and input an exoplanets data in the following:
Lstar = Lsun*10**-0.7450092
Mstar = Msun*0.73
Teq = 720
a = au*0.0372
x0 = 0.5 #Initial WMF
CMFprime = 0.325 #Reduced CMF
Lsat = (10**(-3.5))*Lsun*(Mstar/Msun) #LHE During the saturation phase

***INITIALIZE VARIABLES***

In [106]:
t = 1e6 * yr #1 million years to seconds
M0 = 6.75 * Mearth #Initial mass of Planet
Mp = M0 #initialize the mass
R0 = RpFunction(CMFprime, x, Teq, Mp)
x = x0

***DEFINE FUNCTIONS***

In [96]:
def RpFunction(CMF, x, Teq, Mp): #Function to find radius of planet in m (CMF, WMF, Teq, Mp)
    radius = iop.radius_iop(CMF, x, Teq, Mp/Mearth) #thank you artem for the heavy lifting :D
    return radius*Rearth

def nFunction(Mp, Rp): #N is the efficiency parameter (Greek letter) how much energy is converted to mass loss
    vesc = ((2*G*Mp)/(Rp))**(1/2) #Escape velocity of the planet
    n = 0.1*((vesc/15)**-2)
    return n

def LheFunction(t): #LHE
    if (t<tsat):
        return Lsat
    else:
        Lhe = Lsat*((t/tsat)**(-1-a0))
        return Lhe

def mFunction(Mp, Rp, t): #Time derivative of Mp in kg/s lost
    Lhe = LheFunction(t)
    n = nFunction(Mp, Rp)
    m = (n*(((pi)*(Rp**3)*(Lhe))/(4*(pi)*(a**2)*(G)*(Mp))))
    return m #[(m*1e9*yr)/Mearth] to return mass in Earthmass per billion years


def xwmfFunction(Mp): #xwmf = WMF
    x = (M0*x0-M0+Mp)/Mp
    return x 


***EVOLUTION***

In [114]:
# Start with 1 million years
t = 1e6 * yr
deltat = 1e6 * yr  # Increment by 1 million years

print(f"Starting time {t/yr:.2f} years, Mass {M0/Mearth:.6f} M_earth, Radius {R0/Rearth:.6f} R_earth, WMF: {x0:.6f}, Mass Loss Rate: {m:.6e} kg/s")

iteration = 0  

while (t < (5 * 1e9 * yr)):  # Run until 5 billion years
    t += deltat  # Increment time
    m = mFunction(Mp, Rp, t)  # Update mass loss rate first (depends on current values)
    Mp -= m*deltat  # Now update Mp using the newly computed m
    x = xwmfFunction(Mp)  # Update WMF
    Rp = RpFunction(CMFprime, x, Teq, Mp)  # Update radius
    iteration += 1

    # Print every iteration
    print(f"Iteration {iteration}: Time {t/yr:.2f} years, Mass {Mp/Mearth:.8f} M_earth, Radius {Rp/Rearth:.8f} R_earth, WMF: {x:.9f}, Mass Loss Rate: {m:.6e} kg/s")

print("Evolution loop complete.")  

Starting time 1000000.00 years, Mass 6.750000 M_earth, Radius 2.467746 R_earth, WMF: 0.500000, Mass Loss Rate: 1.927250e-01 kg/s
Iteration 1: Time 2000000.00 years, Mass 6.74999932 M_earth, Radius 2.46774581 R_earth, WMF: 0.499999950, Mass Loss Rate: 6.813856e+01 kg/s
Iteration 2: Time 3000000.00 years, Mass 6.74999932 M_earth, Radius 2.46774581 R_earth, WMF: 0.499999950, Mass Loss Rate: 6.813856e+01 kg/s
Iteration 3: Time 4000000.00 years, Mass 6.74999932 M_earth, Radius 2.46774581 R_earth, WMF: 0.499999950, Mass Loss Rate: 6.813856e+01 kg/s
Iteration 4: Time 5000000.00 years, Mass 6.74999932 M_earth, Radius 2.46774581 R_earth, WMF: 0.499999949, Mass Loss Rate: 6.813856e+01 kg/s
Iteration 5: Time 6000000.00 years, Mass 6.74999932 M_earth, Radius 2.46774581 R_earth, WMF: 0.499999949, Mass Loss Rate: 6.813856e+01 kg/s
Iteration 6: Time 7000000.00 years, Mass 6.74999932 M_earth, Radius 2.46774581 R_earth, WMF: 0.499999949, Mass Loss Rate: 6.813856e+01 kg/s
Iteration 7: Time 8000000.00 ye